In [32]:
#import necessary modules
from bs4 import BeautifulSoup
import requests
from requests.exceptions import Timeout
from requests.exceptions import RequestException
#the function get_product_title fetch the title of the product
def get_product_title(page_soup):
    try:
        product_title = page_soup.find('h1', class_='h2')
        product_title = product_title.text.strip()
        return product_title
    except requests.exceptions.RequestException as e:
        return None
#the function get_product_price fetch the price of the product
def get_product_price(page_soup):
    try:
        
        product_price = page_soup.find('span',id ='productPrice-product-template').text.split('\n')[2]
        return product_price
    except requests.exceptions.RequestException as e:
        return None
#the function get_product_size fetch the size of the product 
def get_product_size(page_soup):
    try:
        
        product_size = [i.text.split('-')[0] for i in page_soup.find_all('option')]
        return product_size
    except requests.exceptions.RequestException as e:
        return None
#the function get_product_size fetch the description of the product
def get_product_desc(page_soup):
    try:
        desc = page_soup.find('div',class_='product-description rte')
        a = [i.text for i in desc.find_all('p')]
        return a
    except requests.exceptions.RequestException as e:
        return None
#i was not able to complete this line of code to fetch the image link of the product
# def get_image(page_soup):
#     try:
        
#         image = page_soup.find('div', class_ = 'no-js product__image-wrapper')
#         a = image.find('img').get('src')
#         return a
#     except requests.exceptions.RequestException as e:
#         return None

#this function retries the connection in case of connection failure error
def get_with_retry(url):
    retries = 3
    timeout = 10  # adjust the timeout value according to your requirements
    for _ in range(retries):
        try:
            response = requests.get(url, timeout=timeout)
            response.raise_for_status()
            return response
        except (Timeout, ConnectionError, RequestException) as e:
            print(f"An error occurred: {e}")
            print(f"Retrying {retries} more times...")
    raise Exception("Max retries exceeded. Failed to fetch URL.")
#this function combines all the function and return a list containing all the data
def scrape_clothing_website(url):
    
    response = get_with_retry(url)
    soup = BeautifulSoup(response.content, "html.parser")
    
    # Find the links to subsequent pages
    page_links = soup.find_all("a", class_="product-grid-item")
    product_list = []
    # Scrape data from each page
    for link in page_links:
        page_url = 'https://www.stylecentrewholesale.co.uk/' +link["href"] 
        #print(page_url)
        page_response = get_with_retry(page_url)
        page_soup = BeautifulSoup(page_response.content, "html.parser")
        product_info = {}
        product_info['title'] = get_product_title(page_soup)
        product_info['price'] = get_product_price(page_soup)
        product_info['size'] = get_product_size(page_soup)
        product_info['description'] = get_product_desc(page_soup)
        #product_info['image link'] = get_image(page_soup)
        
        product_list.append(product_info)
    return product_list
    

# Usage
url = "https://www.stylecentrewholesale.co.uk/collections/all"  # Replace with the URL of the clothing website
l1 = scrape_clothing_website(url)
l1

An error occurred: HTTPSConnectionPool(host='www.stylecentrewholesale.co.uk', port=443): Max retries exceeded with url: //collections/all/products/alife-painkiller-logo-white-t-shirt (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000021DE92BFF10>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Retrying 3 more times...
An error occurred: HTTPSConnectionPool(host='www.stylecentrewholesale.co.uk', port=443): Max retries exceeded with url: //collections/all/products/alife-painkiller-logo-white-t-shirt (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000021DE92BF6D0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Retrying 3 more times...


[{'title': 'Alexander McQueen 526221 WHRUU 1053 Trainers',
  'price': '£400.00',
  'size': ['EU 41 ', 'EU 42 ', 'EU 43 ', 'EU 44 '],
  'description': ['Alexander McQueen 526221 WHRUU 1053 Trainers',
   '-\xa0Alexander McQueen Trainers',
   '- Rubber Sole',
   '- 100% Nappa Leather',
   '- RRP £920',
   '- 56% Discount from RRP']},
 {'title': 'Alexander McQueen Larry 476228 WHGP5 9061 Oversized Trainers',
  'price': '£340.00',
  'size': ['UK 8 ', 'UK 9 '],
  'description': ['Alexander McQueen Larry 476228 WHGP5 9061 Oversized Trainers. Silver metal appliques. Branded print on rear. Black heel tabs. Oversized rubber sole. 100% Calfskin leather, Made In Italy']},
 {'title': 'Alife ALIFW20 13 Navy Blue Casual Shirt',
  'price': '£25.00',
  'size': ['M ', 'XL '],
  'description': ['Alife Baseball Navy Blue Casual Shirt. Alife Baseball Navy Blue Casual Shirt. Mesh Style, Branding Across The Chest. Central Button Closure. Regular Fit, Fits True To Size. Style: ALIFW20 13']},
 {'title': 'Alife

In [12]:
#omport necessary modules
from datetime import datetime
import requests
import csv
import bs4
#these two modules are only for making the scraping faster and i have no idea how they work lol
import concurrent.futures
from tqdm import tqdm
NO_THREADS = 100
if __name__ == "__main__":
    products_data = []
    urls = []
    #open the file containing the links of all the pages(this file contains 30 links)
    with open('links1.csv', newline='') as csvfile:
        #convert it to list
        urls = list(csv.reader(csvfile, delimiter=','))
        print(urls[1][0])
    # for each link in the urls list scrape the links in a loop
    with concurrent.futures.ThreadPoolExecutor(max_workers=NO_THREADS) as executor:
        result1 = []
        for wkn in tqdm(range(1, len(urls))):
            future = executor.submit(scrape_clothing_website, urls[wkn][0])
            result = future.result()
            result1.append(result)
            print(result)


https://www.stylecentrewholesale.co.uk/collections/all


  3%|██▊                                                                                | 1/30 [00:54<26:28, 54.78s/it]

[{'title': 'Alexander McQueen 526221 WHRUU 1053 Trainers', 'price': '£400.00', 'size': ['EU 41 ', 'EU 42 ', 'EU 43 ', 'EU 44 '], 'description': ['Alexander McQueen 526221 WHRUU 1053 Trainers', '-\xa0Alexander McQueen Trainers', '- Rubber Sole', '- 100% Nappa Leather', '- RRP £920', '- 56% Discount from RRP']}, {'title': 'Alexander McQueen Larry 476228 WHGP5 9061 Oversized Trainers', 'price': '£340.00', 'size': ['UK 8 ', 'UK 9 '], 'description': ['Alexander McQueen Larry 476228 WHGP5 9061 Oversized Trainers. Silver metal appliques. Branded print on rear. Black heel tabs. Oversized rubber sole. 100% Calfskin leather, Made In Italy']}, {'title': 'Alife ALIFW20 13 Navy Blue Casual Shirt', 'price': '£25.00', 'size': ['M ', 'XL '], 'description': ['Alife Baseball Navy Blue Casual Shirt. Alife Baseball Navy Blue Casual Shirt. Mesh Style, Branding Across The Chest. Central Button Closure. Regular Fit, Fits True To Size. Style: ALIFW20 13']}, {'title': 'Alife ALIFW20 29 Grey Hoodie', 'price': '

  7%|█████▌                                                                             | 2/30 [01:18<17:04, 36.58s/it]

[{'title': 'Aquascutum 2046 90010701 Red V-Neck Jumper', 'price': '£28.00', 'size': ['L ', 'M ', 'S ', 'XL '], 'description': ['Aquascutum Check Logo Red V-Neck Jumper. Aquascutum Check Logo Red Knitwear V-Neck Sweater. 50% Wool, 50% Acrylic. Branded A In Classic Check On Left Chest. Regular Fit, Fits True To Size. 2046 90010701']}, {'title': 'Aquascutum 2166 01 Blue V-Neck Jumper', 'price': '£30.00', 'size': ['\n                L ', 'M ', 'S '], 'description': ['Aquascutum Check A Logo Medium Blue V-Neck Jumper. Aquascutum Check Logo Medium Blue Knitwear Sweater. 50% Wool, 50% Acrylic. Branded A In Classic Check On Left Chest. Regular Fit, Fits True To Size. 2166 01']}, {'title': 'Aquascutum 28M516 01 Dark Grey V-Neck Jumper', 'price': '£30.00', 'size': ['\n                L ', 'S '], 'description': ['Aquascutum Check A Logo Dark Grey V-Neck Jumper. Aquascutum Check Logo Dark Grey Knitwear Sweater. 50% Wool, 50% Acrylic. Branded A In Classic Check On Left Chest. Regular Fit, Fits True

...

[{'title': 'Cavalli Class QXT64V KB002 03030 Blue Polo Shirt', 'price': '£30.00', 'size': ['L ', 'M ', 'XL ', 'XXL '], 'description': ["Cavalli Class Twinned Tipped Collar Blue Polo Shirt. Step out in style in this Cavalli class blue Polo Shirt. Featuring a black, patch logo, 75% cotton stretch fit, and Cavalli Class' signature blue, this shirt is perfect for any casual occasion. With short sleeves and a regular fit, you'll look sharp and stay comfortable all day.. Black Patch Logo, Short Sleeves, Cavalli Class Blue Shirt. Stretch Fit 75% Cotton, 23% Polyester 2% Elastane. Regular Fit, Fits True To Size. QXT64V KB002 03030"]}, {'title': 'Cavalli Class QXT64V KB002 04511 Avian Polo Shirt', 'price': '£30.00', 'size': ['L ', 'M ', 'XL ', 'XXL '], 'description': ['Cavalli Class Patch Logo Avian Polo Shirt. Look effortlessly stylish in this Cavalli class blue logo white polo shirt. It features a black patch logo, short sleeves, Cavalli Class blue shirt, and a stretch fit. The fabric blend o

 97%|███████████████████████████████████████████████████████████████████████████████▎  | 29/30 [14:04<00:28, 28.99s/it]

[{'title': 'Champion 214188 KK001 Black Jumper', 'price': '£20.00', 'size': ['\n                L ', 'M ', 'S ', 'XL ', '\n                XXL '], 'description': ['Champion Embroidered Script Logo Black Jumper. Champion Black Sweater. 100% Cotton. Elasticated Neck, Sleeve Ends and Bottom. Branded Logo On Chest. Style: 214188 KK001']}, {'title': 'Champion 214394 BS501 Navy Blue Polo Shirt', 'price': '£15.00', 'size': ['\n                L ', '\n                M ', 'S ', '\n                XL '], 'description': ['Champion Red Tipped Collar Navy Blue Polo Shirt. Champion Blue Polo Shirt. Branded Logo On The Left Chest. Regular Fit, Fits True To Size. 100% Cotton. Product Code - 214394 BS501']}, {'title': 'Champion 214405 BS538 Navy T-Shirt', 'price': '£11.00', 'size': ['\n                L ', '\n                M ', 'S ', '\n                XXL ', '\n                XL ', 'XS '], 'description': ['Champion Large C Logo Navy T-Shirt. Champion Blue Tee. Branded Logo On Chest. 100% Cotton. R

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [14:31<00:00, 29.06s/it]

[{'title': 'Champion 214783 GS502 Green Hoodie', 'price': '£25.00', 'size': ['L ', 'M ', 'S '], 'description': ['Champion Colour Block Logo Green Hoodie. Champion Green Hoodie. Elasticated Sleeve Ends and Waist, Drawstring Hood. Front Pockets. 100% Cotton. Style Code: 214783 GS502']}, {'title': 'Champion 214784 KK001 Black Hoodie', 'price': '£25.00', 'size': ['\n                L ', '\n                M ', 'S '], 'description': ['Champion Half Zip Colour Block Logo Black Hoodie. Champion Black Hoodie. Elasticated Sleeve Ends and Waist, Drawstring Hood. Front Pockets. 100% Cotton. Style Code: 214784 KK001']}, {'title': 'Champion 214834 BS508 Navy Blue Hoodie', 'price': '£22.00', 'size': ['\n                L ', '\n                M ', 'S '], 'description': ['Champion Half Zip Large Logo Navy Blue Hoodie. Champion Blue Hoodie. Elasticated Sleeve Ends and Waist, Drawstring Hood. Front Pockets. 100% Cotton. Style Code: 214834 BS508']}, {'title': 'Champion 214834 GS011 Green Hoodie', 'price

In [16]:
import pandas as pd
#as the above code gives result in a list and each item corresponds
#to all the details of a product i decided to convert each item as a dataframe and store it in a list
list1 = []
for i in result1:
    df = pd.DataFrame(i)
    list1.append(df)
#using pd.concat function to to append all the dataframes in the list to a single dataframe
df1 = pd.concat(list1,ignore_index=True)
df1.to_csv('df1.csv',index=False)
df1

,title,price,size,description
0,Alexander McQueen 526221 WHRUU 1053 Trainers,£400.00,"[EU 41 , EU 42 , EU 43 , EU 44 ]","[Alexander McQueen 526221 WHRUU 1053 Trainers,..."
1,Alexander McQueen Larry 476228 WHGP5 9061 Over...,£340.00,"[UK 8 , UK 9 ]",[Alexander McQueen Larry 476228 WHGP5 9061 Ove...
2,Alife ALIFW20 13 Navy Blue Casual Shirt,£25.00,"[M , XL ]",[Alife Baseball Navy Blue Casual Shirt. Alife ...
3,Alife ALIFW20 29 Grey Hoodie,£35.00,"[\n M , S ]",[Alife Collegiate Grey Hoodie. Alife Collegiat...
4,Alife ALIFW20 30 Black Hoodie,£35.00,"[L , M ]",[Alife Collegiate Black Hoodie. Alife Collegia...
...,...,...,...,...
715,Champion 215058 GS028 Green Padded Jacket,£65.00,"[\n L , \n M , \...",[Champion Branded Hooded Dark Green Padded Jac...
716,Champion 215159 KK001 Black Hoodie,£30.00,"[L , \n XXL , \n ...",[Champion Signature Logo Black Hoodie. Champio...
717,Champion 215160 EM004 Grey Jumper,£28.00,"[\n L , M , XL , S ]",[Champion Reverse Weave Classic Script Logo Gr...
718,Champion 215160 KK001 Black Jumper,£28.00,"[L , M , \n XL ]",[Champion Reverse Weave Classic Script Logo Bl...


In [19]:
#repeating the code until all the links are scraped
NO_THREADS = 100
if __name__ == "__main__":
    products_data = []
    urls = []
    with open('links2.csv', newline='') as csvfile:
        urls = list(csv.reader(csvfile, delimiter=','))
        print(urls[1][0])
    with concurrent.futures.ThreadPoolExecutor(max_workers=NO_THREADS) as executor:
        result2 = []
        for wkn in tqdm(range(1, len(urls))):
            future = executor.submit(scrape_clothing_website, urls[wkn][0])
            result = future.result()
            result2.append(result)
            print(result)

https://www.stylecentrewholesale.co.uk/collections/all?page=31


  3%|██▊                                                                                | 1/30 [00:25<12:28, 25.82s/it]

[{'title': 'Champion 215215 BS501 Blue Sweatshirt', 'price': '£28.00', 'size': ['\n                L ', '\n                M ', 'S ', '\n                XL ', 'XS '], 'description': ['Champion Small Logo Navy Blue Sweatshirt. Champion Blue Crew Neck Jumper. Branded Logo On Chest. 86% Cotton 14% Polyester. Elasticated Sleeve and Waist Endings. Product Code - 215215 BS501']}, {'title': 'Champion 215249 BS538 Hood Navy Blue Jacket', 'price': '£38.00', 'size': ['L ', 'M ', 'S ', '\n                XL ', 'XS ', 'XXL '], 'description': ['Champion Hidden Hood Navy Blue Jacket. Champion Blue Jacket. Branded Logo On Sleeve. 100% Polyamide. Zip Closure, Elasticated Cuff and Hem. Product Code - 215249 BS538']}, {'title': 'Champion 215249 GS505 Hood Dark Green Jacket', 'price': '£38.00', 'size': ['L ', 'M ', 'XL ', 'XS ', 'XXL ', 'S '], 'description': ['Champion Hidden Hood Dark Green Jacket. Champion Green Jacket. Branded Logo On Sleeve. 100% Polyamide. Zip Closure, Elasticated Cuff and Hem. Prod

  7%|█████▌                                                                             | 2/30 [00:52<12:09, 26.07s/it]

[{'title': 'Champion 216961 WW001 White Hoodie', 'price': '£22.00', 'size': ['L ', 'M ', 'S ', '\n                XL ', 'XS ', '\n                XXL '], 'description': ['Champion Digital Print Logo White Hoodie. Champion White Hoodie. Elasticated Sleeve Ends and Waist, Drawstring Hood. Front Pockets. 73% Cotton 27% Polyester. Style Code: 216961 WW001']}, {'title': 'Champion 217606 KK004 Black Hoodie', 'price': '£28.00', 'size': ['\n                L ', '\n                M ', 'S ', 'XS '], 'description': ['Champion x Muhammad Ali Logo Black Hoodie. Champion Black Hoodie. Elasticated Sleeve Ends and Waist, Drawstring Hood. Front Pockets. 84% Cotton 16% Polyester. Style Code: 217606 KK004']}, {'title': 'Champion 217611 KK004 Black Hoodie', 'price': '£22.00', 'size': ['L ', 'M ', 'XL ', 'XXL ', 'S ', 'XS '], 'description': ['Champion Rome Logo Black Hoodie. Champion Black Hoodie. Elasticated Sleeve Ends and Waist, Drawstring Hood. Front Pockets. 84% Cotton 16% Polyester. Style Code: 2176

...

[{'title': 'Dsquared2 SNM0199 01602625 M2374 White Sneakers', 'price': '£170.00', 'size': ['7 ', '7.5 '], 'description': ['Dsquared2 Green Striped White Sneakers. Be the envy of your friends in these stylish and comfortable Dsquared2 White Sneakers! With timeless style and cutting-edge materials, these sneakers are perfect for anyone looking to step up their fashion game. Get ready to take your street style to the next level!. Dsquared2 White Trainers, Low Cut Sneaker, Made In Italy. Rubber Sole, 100% Leather Upper. Tape Logo Branding. SNM0199 01602625 M2374']}, {'title': 'Dsquared2 Super Twinky S74LB0810 S30503 900 Jeans', 'price': '£180.00', 'size': ['48 / 30L ', '\n                50 / 30L ', '46 / 30L '], 'description': ['Dsquared2 Super Twinky Jean Paint Splash Destroyed Jeans. Super Twinky Jean S74LB0810 S30503 900. Stretch Denim 92% Cotton, 6% Elastomultiestere, 2% Elastane. Button Fly, Skinny Fit With A Tapered Leg. Paint Splash Detail, Destroyed Reinforced Denim. Made In Italy

 97%|███████████████████████████████████████████████████████████████████████████████▎  | 29/30 [13:45<00:28, 28.61s/it]

[{'title': 'EA7 3HPM82 PJP7Z 1570 Blue Zip Up Hoodie', 'price': '£65.00', 'size': ['L ', 'M ', '\n                S ', 'XL ', 'XXL ', 'XXXL '], 'description': ['EA7 Metal Chest Logo Blue Zip Up Hoodie. EA7 Blue Hoodie. Style Code: 3HPM82 PJP7Z 1570. 95% Cotton, 5% Elastane. Ribbed Sleeve and Waist Endings. Drawstring Ties']}, {'title': 'EA7 3HPT21 PJ8CZ 1900 Navy Blue T-Shirt', 'price': '£25.00', 'size': ['\n                XL ', 'XXL '], 'description': ['EA7 Brand Crest Chest Logo Navy Blue T-Shirt. EA7 Grey T-Shirt. Style Code: 3HPT21 PJ8CZ 1900. 100% Polyester. Crew Neck. Logo Down Left Side Of Front Tee']}, {'title': 'EA7 3HPT35 PJ7BZ 1595 Blue Skydiver T-Shirt', 'price': '£27.00', 'size': ['XXL '], 'description': ['EA7 Chest Logo Blue Skydiver T-Shirt. EA7 Blue T-Shirt. Style Code: 3HPT35 PJ7BZ 1595. Crew Neck. 100% Cotton. Stripe Pattern On Shoulders']}, {'title': 'EA7 3HPT85 PJM9Z 1200 Black T-Shirt', 'price': '£28.00', 'size': ['L ', '\n                M '], 'description': ['EA

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [14:14<00:00, 28.49s/it]

[{'title': 'Emporio Armani 3Z1BQ3 1NFXZ  F900 Jacket', 'price': '£125.00', 'size': ['\n                EU 48 ', '\n                EU 50 ', '\n                EU 52 ', '\n                EU 54 ', 'EU 56 '], 'description': ['Emporio Armani 3Z1BQ3 1NFXZ  F900 Jacket. Emporio Armani Navy Blue Jacket. Zip Closure. 100% Polyester. Zipped Pockets. Branded Badge On Left Arm']}, {'title': 'Emporio Armani 3Z1T77 0544 T-Shirt', 'price': '£28.00', 'size': ['XS ', 'S ', '\n                M ', 'L ', '\n                XL '], 'description': ['Emporio Armani 3Z1T77 0544 T-Shirt. Short Sleeved Green T-Shirt. Logo On Front Bottom Left. 100% Cotton. Style: 3Z1T77 1JPZZ. V - Neck']}, {'title': 'Emporio Armani 3Z1T77 0620 T-Shirt', 'price': '£28.00', 'size': ['XS ', 'S ', '\n                M ', '\n                L ', '\n                XXL ', '\n                XL '], 'description': ['Emporio Armani 3Z1T77 0620 T-Shirt. Short Sleeved Grey T-Shirt. Logo On Front Bottom Left. 100% Cotton. Style: 3Z1T77 1

In [20]:
list2 = []
for i in result2:
    df = pd.DataFrame(i)
    list2.append(df)
df2 = pd.concat(list2,ignore_index=True)
df2.to_csv('df2.csv',index=False)
df2

,title,price,size,description
0,Champion 215215 BS501 Blue Sweatshirt,£28.00,"[\n L , \n M , S...",[Champion Small Logo Navy Blue Sweatshirt. Cha...
1,Champion 215249 BS538 Hood Navy Blue Jacket,£38.00,"[L , M , S , \n XL , XS , XXL ]",[Champion Hidden Hood Navy Blue Jacket. Champi...
2,Champion 215249 GS505 Hood Dark Green Jacket,£38.00,"[L , M , XL , XS , XXL , S ]",[Champion Hidden Hood Dark Green Jacket. Champ...
3,Champion 215249 KK001 Hood Black Jacket,£38.00,"[M , S , \n XS , \n ...",[Champion Hidden Hood Black Jacket. Champion B...
4,Champion 215283 WW001 White Hoodie,£25.00,"[S , L , M ]",[Champion Small Classic Script Logo White Hood...
...,...,...,...,...
715,Emporio Armani 8N1F13 1J0SZ 0999 Polo Shirt,£42.00,"[XXS , \n M , \n ...",[Emporio Armani 8N1F13 1J0SZ 0999 Polo Shirt. ...
716,Emporio Armani AR0382 Two Hand Black Leather S...,£80.00,[Default Title ],[]
717,Emporio Armani AR11121 Three Hand Two Tone Sil...,£70.00,[Default Title ],[]
718,Emporio Armani AR11324 Luigi Chronograph Silve...,£90.00,[Default Title ],[]


In [21]:
NO_THREADS = 100
if __name__ == "__main__":
    products_data = []
    urls = []
    with open('links3.csv', newline='') as csvfile:
        urls = list(csv.reader(csvfile, delimiter=','))
        print(urls[1][0])
    with concurrent.futures.ThreadPoolExecutor(max_workers=NO_THREADS) as executor:
        result3 = []
        for wkn in tqdm(range(1, len(urls))):
            future = executor.submit(scrape_clothing_website, urls[wkn][0])
            result = future.result()
            result3.append(result)
            print(result)
            

https://www.stylecentrewholesale.co.uk/collections/all?page=61


  3%|██▊                                                                                | 1/30 [00:28<13:56, 28.83s/it]

[{'title': 'Emporio Armani AR1853 Chronograph Silver Watch', 'price': '£90.00', 'size': ['Default Title '], 'description': []}, {'title': 'Emporio Armani AR1917 Chronograph Black Leather Strap Watch', 'price': '£80.00', 'size': ['Default Title '], 'description': []}, {'title': 'Emporio Armani AR70003 Black Watch', 'price': '£130.00', 'size': ['Default Title '], 'description': []}, {'title': 'Emporio Armani Chronograph AR11132 Silver Watch', 'price': '£90.00', 'size': ['Default Title '], 'description': []}, {'title': 'Emporio Armani Court Perforated Logo Leather Black Trainers', 'price': '£75.00', 'size': ['10 ', '11 ', '\n                12 ', '\n                7 ', '8 ', '\n                9 '], 'description': ['Emporio Armani Court Perforated Logo Leather Black Trainers. Emporio Armani Court Perforated Logo Leather Black Trainers. Style: X4X316 XM500 N024. Milano Tongue Print, Perforated Eagle Design. Lace Fasten. Leather Upper, Rubber Sole']}, {'title': 'Emporio Armani Court Perfor

  7%|█████▌                                                                             | 2/30 [00:58<13:33, 29.04s/it]

[{'title': 'Fred Perry B1261 102 Black Trainers', 'price': '£45.00', 'size': ['UK 5 ', 'UK 11 ', 'UK 3 ', 'UK 4 ', 'UK 6 ', '\n                UK 9 ', 'UK 9.5 '], 'description': ['Fred Perry B1261 102 Black Trainers. Fred Perry Black Shoes. Style: B1261 102. 100% Leather. Lace Fasten Trainers. Branded Badge On Side Of Shoe']}, {'title': 'Fred Perry B1263 254 White Trainers', 'price': '£40.00', 'size': ['\n                UK 11 ', 'UK 7 ', '\n                UK 8 ', '\n                UK 9.5 '], 'description': ['Fred Perry B1263 254 White Trainers. Fred Perry White Shoes. Style: B1263 254. 100% Leather. Lace Fasten Trainers. Branded Badge On Side Of Shoe']}, {'title': 'Fred Perry B1263 L35 Blue Trainers', 'price': '£30.00', 'size': ['UK 3 ', 'UK 4 ', 'UK 5 ', 'UK 6 '], 'description': ['Fred Perry B1263 L35 Blue Trainers. Fred Perry Blue Shoes. Style: B1263 L35. 100% Leather. Lace Fasten Trainers. Branded Badge On Side Of Shoe']}, {'title': 'Fred Perry B1271 102 Black Leather Trainers', 

...

[{'title': 'Moschino V1706 8120 0489 Grey Jumper', 'price': '£70.00', 'size': ['L '], 'description': ['Moschino Tape Logo Grey Jumper. Moschino Tape Logo Grey Sweater. Elasticated Hem and Cuffs, Crew Neck Jumper. Branded Tape Along The Arms, Comfortable Material 95% Cotton, 5% Elastane. Moschino Underwear Collection, Regular Fit, Fits True To Size. Style Code: V1706 8120 0489']}, {'title': 'Moschino V1724 8129 0430 Green Sweatshirt', 'price': '£50.00', 'size': ['L ', 'M ', 'XL ', 'XXL '], 'description': ['Moschino Panel Brand Logo Green Sweatshirt. Moschino Green Jumper. Panel Logo On Chest, Elasticated Sleeve Ends and Waist. 54% Polyamide, 46% Cotton, Moschnio Underwear Collection. Elasticated Sleeve and Waist Endings. Style Code: V1724 8129 0430']}, {'title': 'Moschino V1724 8129 0555 Black Sweatshirt', 'price': '£50.00', 'size': ['L ', 'M ', 'XL ', 'XXL '], 'description': ['Moschino Panel Brand Logo Black Sweatshirt. Moschino Black Jumper. Panel Logo On Chest, Elasticated Sleeve End

 97%|███████████████████████████████████████████████████████████████████████████████▎  | 29/30 [14:06<00:28, 28.46s/it]

[{'title': 'Napapijri NP0A4EJT0411 Black Jacket', 'price': '£110.00', 'size': ['XL ', 'XXXL ', 'XXL '], 'description': ['Napapijri A-Alvar Black Jacket. Napapijri Black Jacket. Style Code: A-ALVAR NP0A4EJT0411. Reversible. Regular Fit. Zip Closure']}, {'title': 'Napapijri NP0A4EKA1761 Navy Blue Jacket', 'price': '£80.00', 'size': ['M ', 'S '], 'description': ['Napapijri ZE-K304 Navy Blue Jacket. Napapijri ZE-K304 Navy Blue Jacket. Adjustable Toggle Hood, Branded Badge On The Left Shoulder. Regular Fit, Fits True To Size. Zip Twin Pockets, Partially Quilted, Hybrid Jacket Digitally Knitted With A Single Yarn. Style Code: ZE-K304 NP0A4EKA1761']}, {'title': 'Napapijri NP0A4EMK0411 Black Hoodie', 'price': '£50.00', 'size': ['L ', 'XL ', 'XXL '], 'description': ['Napapijri Toli Chest Logo Black Hoodie. Napapijri Black Hoodie. Style Code: TOLI NP0A4EMK0411. 96% Polyester, 4% Elastane. Elasticated Sleeve and Waist Endings. Napariji Branding']}, {'title': 'Napapijri NP0A4ENLGE41 Green Depths J

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [14:33<00:00, 29.10s/it]

[{'title': 'Napapijri NP0A4FA21761 Navy Blue Polo Shirt', 'price': '£25.00', 'size': ['M ', '\n                XL ', 'L ', 'S '], 'description': ['Napapijri ELBAS 4 Logo Navy Blue Polo Shirt. Napapijri Blue Polo Shirt. Style Code: ELBAS 4 NP0A4FA21761. Button Closure At The Neck. Napariji Flag Logo On Chest. 100% Cotton']}, {'title': 'Napapijri NP0A4FA40021 Navy Polo Shirt', 'price': '£30.00', 'size': ['L ', 'M ', 'S ', 'XL '], 'description': ['Napapijri Ellar Large Brand Logo Navy Polo Shirt. Napapijri White Polo Shirt. Style Code: ELLAR NP0A4FA40021. Button Closure At The Neck. Napariji Geography Design on Polo. 100% Cotton']}, {'title': 'Napapijri NP0A4FA40021 White Polo Shirt', 'price': '£28.00', 'size': ['L ', 'M ', 'S ', 'XL '], 'description': ['Napapijri EOLANOS 2 Flag Logo White Polo Shirt. Napapijri White Polo Shirt. Style Code: ELLAR NP0A4FA40021. Button Closure At The Neck. Napariji Geography Design on Polo. 100% Cotton']}, {'title': 'Napapijri NP0A4FA40021 White Polo Shirt'

In [22]:
list3 = []
for i in result3:
    df = pd.DataFrame(i)
    list3.append(df)
df3 = pd.concat(list3,ignore_index=True)
df3.to_csv('df3.csv',index=False)
df3

,title,price,size,description
0,Emporio Armani AR1853 Chronograph Silver Watch,£90.00,[Default Title ],[]
1,Emporio Armani AR1917 Chronograph Black Leathe...,£80.00,[Default Title ],[]
2,Emporio Armani AR70003 Black Watch,£130.00,[Default Title ],[]
3,Emporio Armani Chronograph AR11132 Silver Watch,£90.00,[Default Title ],[]
4,Emporio Armani Court Perforated Logo Leather B...,£75.00,"[10 , 11 , \n 12 , \n ...",[Emporio Armani Court Perforated Logo Leather ...
...,...,...,...,...
715,Napapijri NP0AE7X0411 Black Jacket,£150.00,"[XL , XXL , L , M , S ]",[Napapijri Rainforest Infinity Black Jacket. N...
716,Napapijri Womens NP0A4E57GW11 Mango Yellow Jacket,£75.00,[\n XL ],[Napapijri Womens Rainforest S W PKT 1 Mango Y...
717,Neil Barrett BCT297 H9011 1906 Sneakers,£120.00,"[8 , \n 9 ]",[Neil Barrett Retro Runner Sneakers. Neil Barr...
718,Neil Barrett Big Bolt Sock Skater Sneakers,£110.00,[UK 8 ],[Neil Barrett Big Bolt Sock Skater Sneakers. N...


,title,price,size,description
0,Napapijri NP0A4FA21761 Navy Blue Polo Shirt,£25.00,"[M , \n XL , L , S ]",[Napapijri ELBAS 4 Logo Navy Blue Polo Shirt. ...
1,Napapijri NP0A4FA40021 Navy Polo Shirt,£30.00,"[L , M , S , XL ]",[Napapijri Ellar Large Brand Logo Navy Polo Sh...
2,Napapijri NP0A4FA40021 White Polo Shirt,£28.00,"[L , M , S , XL ]",[Napapijri EOLANOS 2 Flag Logo White Polo Shir...
3,Napapijri NP0A4FA40021 White Polo Shirt,£30.00,"[L , M , S , XL ]",[Napapijri Ellar Large Brand Logo White Polo S...
4,Napapijri NP0A4FDNB2E1 Blue Jacket,£70.00,"[\n L , XL , \n ...",[Napapijri Rainforest M Sum 2 Blue Jacket. Nap...
5,Napapijri NP0A4FDNB2E1 Old Red Jacket,£70.00,"[\n L , \n M , S...",[Napapijri Rainforest M Sum 2 Old Red Jacket. ...
6,Napapijri NP0A4FF50021 Bright White T-Shirt,£16.00,"[\n M , \n S , \...",[Napapijri S-BOX SS 1 Box Logo Bright White T-...
7,Napapijri NP0A4FG80411 Black T-Shirt,£16.00,"[L , M , S , XL , XXL ]",[Napapijri S-SURF SS Logo Black T-Shirt. Napap...
8,Napapijri NP0A4FG80411 Black T-Shirt,£16.00,"[\n L , \n M , \...",[Napapijri S-Patch SS 1 Patch Logo Black T-Shi...
9,Napapijri NP0A4FG80411 Blue T-Shirt,£16.00,"[L , S , XL , \n XXL , M ]",[Napapijri S-SURF SS Logo Medieval Blue T-Shir...


In [33]:
NO_THREADS = 100
if __name__ == "__main__":
    products_data = []
    urls = []
    with open('links4.csv', newline='') as csvfile:
        urls = list(csv.reader(csvfile, delimiter=','))
        print(urls[1][0])
    with concurrent.futures.ThreadPoolExecutor(max_workers=NO_THREADS) as executor:
        result4 = []
        for wkn in tqdm(range(1, len(urls))):
            future = executor.submit(scrape_clothing_website, urls[wkn][0])
            result = future.result()
            result4.append(result)
            print(result)

https://www.stylecentrewholesale.co.uk/collections/all?page=91


  3%|██▌                                                                                | 1/33 [00:22<12:12, 22.90s/it]

[{'title': 'Neil Barrett BJS010S R517S 526 White Sweatshirt', 'price': '£130.00', 'size': ['M ', 'S '], 'description': ['Neil Barrett Fair Isle Thunderbolt Oversize White Sweatshirt. Neil Barrett Fair Isle Thunderbolt Oversize White Sweatshirt. Made In Italy, Oversize Style. Elasticated Neck, Sleeve Ends and Bottom. Signature Thunderbolt Branding. Style: BJS010S R517S 526']}, {'title': 'Neil Barrett BJS465D L574S 1895 Black Sweatshirt', 'price': '£150.00', 'size': ['M ', 'XS '], 'description': ['Neil Barrett Flower Lightning Bolt Black Sweatshirt. Neil Barrett BJS465D L574S 1895 Jumper. 100% Cotton. Regular Fit, Fits True To Size. Made In Italy. Elasticated Neck, Sleeve Ends and Bottom']}, {'title': 'Neil Barrett BJS556S M597S 1408 Green Zip Hoodie', 'price': '£160.00', 'size': ['L '], 'description': ['Neil Barrett Thunderbolt Sleeve Green Zip Hoodie. Neil Barrett BJS556S M597S 1408 Zip Up Hoodie. 95% Viscose 5% Elastane. White Printed Thunderbolt Branding. Regular Fit, Fits True To Si

  6%|█████                                                                              | 2/33 [00:46<11:58, 23.18s/it]

[{'title': 'Neil Barrett PBJT554S M525S 607 Tee', 'price': '£60.00', 'size': ['\n                M ', 'S '], 'description': ['Neil Barrett Killer Bear White Tee. Neil Barrett Killer Bear White T-Shirt. Regular Loose Fit. 50% Cotton, 50% Modal. Made In Italy. PBJT554S M525S 607']}, {'title': 'Neil Barrett XJT236 1AU 01 Black Barrett T-Shirt', 'price': '£50.00', 'size': ['S ', 'XS '], 'description': ['Neil Barrett Actions Speak Louder Than Words Black Barrett T-Shirt. Neil Barrett Actions Speak Louder Than Words Black Barrett T-Shirt. Regular Loose Fit. 50% Cotton, 50% Modal. Large Printed Slogan. XJT236 1AU 01']}, {'title': 'New Balance 373 Suede and Mesh Grey Trainers', 'price': '£32.00', 'size': ['UK 10.5 '], 'description': ['New Balance 373 Suede and Mesh Grey Trainers. New Balance 373 Suede And Mesh Grey Trainers. Branding On The Side, Back and Tongue. 100% Textile, Vulcanised Rubber Sole, Lace Fasten. Textile Lining, Suede And Mesh Upper. Style: 373 Grey']}, {'title': 'New Balance 

...

[{'title': 'Tom Ford Geraldine FT0568 69T Brown Sunglasses', 'price': '£85.00', 'size': ['Default Title '], 'description': []}, {'title': 'Tom Ford Gerard-02 FT0930-N 01D Black Sunglasses', 'price': '£100.00', 'size': ['Default Title '], 'description': []}, {'title': 'Tom Ford Ginger FT0873 52F Brown Sunglasses', 'price': '£85.00', 'size': ['Default Title '], 'description': []}, {'title': 'Tom Ford Jet FT0734-H 01N Sunglasses', 'price': '£80.00', 'size': ['Default Title '], 'description': []}, {'title': 'Tom Ford Joni FT0905 54V Brown Sunglasses', 'price': '£100.00', 'size': ['\n                Default Title '], 'description': []}, {'title': 'Tom Ford Kira FT0821 69T Red Sunglasses', 'price': '£100.00', 'size': ['Default Title '], 'description': []}, {'title': 'Tom Ford Lauren-02 FT0614-F 01C Sunglasses', 'price': '£85.00', 'size': ['Default Title '], 'description': []}, {'title': 'Tom Ford Leah FT0849 01B Black Sunglasses', 'price': '£100.00', 'size': ['Default Title '], 'description'

 97%|███████████████████████████████████████████████████████████████████████████████▌  | 32/33 [12:35<00:23, 23.37s/it]

[{'title': 'Tom Ford Russel FT0795-H 01A Sunglasses', 'price': '£100.00', 'size': ['Default Title '], 'description': []}, {'title': 'Tom Ford Russel-02 FT0795-H 30E Sunglasses', 'price': '£100.00', 'size': ['Default Title '], 'description': []}, {'title': 'Tom Ford Shelby-02 FT0620 08V Black Sunglasses', 'price': '£90.00', 'size': ['Default Title '], 'description': []}, {'title': 'Tom Ford Simone FT0571 28B Rose Gold Sunglasses', 'price': '£85.00', 'size': ['Default Title '], 'description': []}, {'title': 'Tom Ford Stevie FT0761 28Y Black Sunglasses', 'price': '£85.00', 'size': ['Default Title '], 'description': []}, {'title': 'Tom Ford Sye FT0738 01B Black Sunglasses', 'price': '£85.00', 'size': ['\n                Default Title '], 'description': []}, {'title': 'Tom Ford Sye FT0738 28B Rose Gold Sunglasses', 'price': '£85.00', 'size': ['Default Title '], 'description': []}, {'title': 'Tom Ford Tammy FT0770 01B Black Sunglasses', 'price': '£85.00', 'size': ['Default Title '], 'descrip

100%|██████████████████████████████████████████████████████████████████████████████████| 33/33 [12:59<00:00, 23.62s/it]

[{'title': 'True Religion Geno Slim Blue Jeans', 'price': '£40.00', 'size': ['W32 / 34L ', 'W33 / 34L ', 'W34 / 34L ', 'W36 / 34L '], 'description': ['True Religion Geno Slim Blue Jeans. True Religion Geno No Flap 106695 Medium Ink Blue Jeans. Zip Fly With Button Fasten, Faded Blue Denim Style. 98% Cotton 2% Elastane Stretch Denim, Large Branded Badge On Back Right Waist. Stitched Horseshoe Branding On Back Pockets. Style: 106695 Medium Ink_Blue']}, {'title': 'True Religion Horseshoe Puff Jogger Black Sweatpants', 'price': '£30.00', 'size': ['XL ', 'XXL '], 'description': ['True Religion Horseshoe Puff Jogger Black Sweatpants. True Religion Horseshoe Puff Jogger Black Sweatpants. Signature Horseshoe Brand Logo On Front Left Thigh. Drawstring Ties, Elasticated Cuffs and Waist, 2 Side Pockets. Regular Fit, Fits True To Size, Back Pocket With Sticthed Horseshoe Detail. Style Code: 106140 Black']}, {'title': 'True Religion Horseshoe Puff Jogger Grey Sweatpants', 'price': '£30.00', 'size': 

In [34]:
list4 = []
for i in result4:
    df = pd.DataFrame(i)
    list4.append(df)
df4 = pd.concat(list4,ignore_index=True)
df4.to_csv('df4.csv',index=False)
df4

,title,price,size,description
0,Neil Barrett BJS010S R517S 526 White Sweatshirt,£130.00,"[M , S ]",[Neil Barrett Fair Isle Thunderbolt Oversize W...
1,Neil Barrett BJS465D L574S 1895 Black Sweatshirt,£150.00,"[M , XS ]",[Neil Barrett Flower Lightning Bolt Black Swea...
2,Neil Barrett BJS556S M597S 1408 Green Zip Hoodie,£160.00,[L ],[Neil Barrett Thunderbolt Sleeve Green Zip Hoo...
3,Neil Barrett BJS556S M597S 524 Black Zip Hoodie,£275.00,[M ],[Neil Barrett Thunderbolt Sleeve Black Zip Hoo...
4,Neil Barrett BJT006S R514S 524 Black T-Shirt,£90.00,"[\n L , M , S ]",[Neil Barrett Chalk Bolt Black T-Shirt. Neil B...
...,...,...,...,...
787,Valentino 1Y2S0723TCCW42 44 Army Green Sneakers,£300.00,[10 ],[Valentino Low Rockrunner Army Green Sneakers....
788,Valentino 2V3MG13D96S TTL Black T-Shirt,£200.00,"[L , M , S , XL ]",[Valentino VLTN Print College Logo Black T-Shi...
789,Valentino Garavani Rock Runner Green Camo Snea...,£255.00,[8 ],[Valentino Garavani Rock Runner Green Camo Sne...
790,Valentino NY0S0A28 HTH 0NO leather Black Sneakers,£200.00,[\n 8 ],[Valentino Garavani Panelled leather Black Sne...


In [1]:
#finally concanetating all the data into a single dataframe and doing some cleaning

df_new = pd.concat([df,df2,df3,df4],ignore_index=True)
df_main['category'] = df_main['title'].apply(lambda x: x.split()[-1])
df_main['color'] = df_main['title'].apply(lambda x : x.split()[-2])
df_main.columns = ['TITLE','PRICE','SIZE','DESCRIPTION','CATEGORY','COLOR']
df_main = df_main[['TITLE','CATEGORY','COLOR','SIZE','PRICE','DESCRIPTION']]
df_main.to_csv('project.csv',index=False)

NameError: name 'pd' is not defined